In [ ]:
# Install necessary libraries
!pip install monai torch torchvision
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 44.1 MB/s eta 0:00:00


In [ ]:
!pip install nibabel

In [ ]:
!unzip "/content/drive/MyDrive/MAIA_Work/Semester_3/MISA/MISA_Project/TABS.zip" -d /content/

Archive:  /content/drive/MyDrive/MAIA_Work/Semester_3/MISA/MISA_Project/TABS.zip
   creating: /content/TABS/.git/
  inflating: /content/TABS/.git/config  
  inflating: /content/TABS/.git/description  
 extracting: /content/TABS/.git/HEAD  
   creating: /content/TABS/.git/hooks/
  inflating: /content/TABS/.git/hooks/applypatch-msg.sample  
  inflating: /content/TABS/.git/hooks/commit-msg.sample  
  inflating: /content/TABS/.git/hooks/fsmonitor-watchman.sample  
  inflating: /content/TABS/.git/hooks/post-update.sample  
  inflating: /content/TABS/.git/hooks/pre-applypatch.sample  
  inflating: /content/TABS/.git/hooks/pre-commit.sample  
  inflating: /content/TABS/.git/hooks/pre-merge-commit.sample  
  inflating: /content/TABS/.git/hooks/prepare-commit-msg.sample  
  inflating: /content/TABS/.git/hooks/pre-push.sample  
  inflating: /content/TABS/.git/hooks/pre-rebase.sample  
  inflating: /content/TABS/.git/hooks/pre-receive.sample  
  inflating: /content/TABS/.git/hooks/push-to-checkou

In [ ]:
import sys
sys.path.append('/content/TABS')

In [ ]:
from monai.transforms import (
    Compose, LoadImage, Spacing, NormalizeIntensity, ScaleIntensityRange, CenterSpatialCrop
)
from torch.utils.data import DataLoader, Dataset
import nibabel as nib
import numpy as np
import os
import torch
# from TABS.Models.TABS_Model import TABS
from monai.losses import DiceCELoss
from torch.optim import AdamW
from torch.cuda.amp import GradScaler, autocast

In [ ]:
!nvidia-smi

Mon Dec 30 14:46:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              47W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
%cd /content/drive/MyDrive/MAIA_Work/Semester_3/MISA/MISA_Project/TABS
!python train.py --root /content/drive/MyDrive/MAIA_Work/Semester_3/MISA/MISA_Project/TABS --gpu 0

/content/drive/MyDrive/MAIA_Work/Semester_3/MISA/MISA_Project/TABS
Using GPU: NVIDIA A100-SXM4-40GB
/content/drive/MyDrive/MAIA_Work/Semester_3/MISA/MISA_Project/TABS/train.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub fo

In [ ]:
# # Image preprocessing pipeline
# image_transforms = Compose([
#     LoadImage(image_only=True, ensure_channel_first=True),
#     Spacing(pixdim=(1.0, 1.0, 1.0), mode='bilinear'),
#     CenterSpatialCrop(roi_size=(192, 192, 192)),
#     NormalizeIntensity(nonzero=True, channel_wise=True),
#     ScaleIntensityRange(a_min=-3.364, a_max=1.81, b_min=-1.0, b_max=1.0, clip=True),
# ])

# # Label preprocessing pipeline
# label_transforms = Compose([
#     LoadImage(image_only=True, ensure_channel_first=True),
#     Spacing(pixdim=(1.0, 1.0, 1.0), mode='nearest'),
#     CenterSpatialCrop(roi_size=(192, 192, 192)),
# ])

In [ ]:
class BrainSegmentationDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        """
        Args:
            image_dir (str): Path to the directory containing preprocessed images.
            label_dir (str): Path to the directory containing preprocessed labels.
            transform (callable, optional): Transform to apply to the data. Defaults to None.
        """
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.image_filenames = sorted(os.listdir(image_dir))
        self.label_filenames = sorted(os.listdir(label_dir))
        self.transform = transform

        assert len(self.image_filenames) == len(self.label_filenames), \
            "Number of images and labels must match."

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        # Load the image and label file paths
        image_path = os.path.join(self.image_dir, self.image_filenames[idx])
        label_path = os.path.join(self.label_dir, self.label_filenames[idx])

        # Load image and label data
        image = nib.load(image_path).get_fdata().astype(np.float32)  # Shape: (C, D, H, W)
        label = nib.load(label_path).get_fdata().astype(np.int64)    # Shape: (C, D, H, W)

        # Apply transforms to the image (if any)
        if self.transform:
            image = self.transform(image)

        # Convert to PyTorch tensors
        image = torch.tensor(image, dtype=torch.float32)  # Shape: [C, D, H, W]
        label = torch.tensor(label, dtype=torch.float32)     # Shape: [C, D, H, W]

        return image, label

In [ ]:
# Paths to dataset
train_image_dir = "/content/drive/MyDrive/MAIA_Work/Semester_3/MISA/MISA_Project/ProcessedDataV3/Training_Set/Image"
train_label_dir = "/content/drive/MyDrive/MAIA_Work/Semester_3/MISA/MISA_Project/ProcessedDataV3/Training_Set/Label_ProbMaps"

# Create Datasets and DataLoaders
train_dataset = BrainSegmentationDataset(train_image_dir, train_label_dir)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)

In [ ]:
# Paths to dataset
valid_image_dir = "/content/drive/MyDrive/MAIA_Work/Semester_3/MISA/MISA_Project/ProcessedDataV3/Validation_Set/Image"
valid_label_dir = "/content/drive/MyDrive/MAIA_Work/Semester_3/MISA/MISA_Project/ProcessedDataV3/Validation_Set/Label_ProbMaps"

# Create Datasets and DataLoaders
valid_dataset = BrainSegmentationDataset(valid_image_dir, valid_label_dir)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=True, num_workers=0)

In [ ]:
# Example: Iterate through the DataLoader
for images, labels in train_loader:
    print(f"Image shape: {images.shape}")  # (batch_size, 1, 192, 192, 192)
    print(f"Label shape: {labels.shape}")  # (batch_size, 1, 192, 192, 192)
    break

Image shape: torch.Size([4, 1, 192, 192, 192])
Label shape: torch.Size([4, 3, 192, 192, 192])


In [ ]:
# num_epochs = 50
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load the pretrained TABS model
# model = TABS()
# pretrained_weights_path = '/content/drive/MyDrive/MAIA_Work/Semester_3/MISA/MISA_Project/best_model_TABS.pth'
# checkpoint = torch.load(pretrained_weights_path, map_location=device)
# model.load_state_dict(checkpoint['state_dict'])  # Adjust if checkpoint key differs
# model.to(device)

# # Optimizer, scaler, and loss function
# optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
# scaler = GradScaler()
# criterion = DiceCELoss(include_background=False, softmax=True)  # Use your preferred loss function

In [ ]:
# for epoch in range(num_epochs):
#     model.train()
#     train_loss = 0

#     for images, labels in train_loader:
#         images, labels = images.to(device), labels.to(device)

#         # Ensure labels are in correct format
#         labels = labels.squeeze(1)  # [batch_size, D, H, W]

#         optimizer.zero_grad()
#         with autocast(device_type='cuda'):
#             outputs = model(images)  # [batch_size, num_classes, D, H, W]
#             loss = criterion(outputs, labels)

#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()

#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)
#     print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}")

#     # Save model checkpoint
#     checkpoint_path = f"/content/drive/MyDrive/MAIA_Work/Semester_3/MISA/MISA_Project/TABS_finetuned_epoch_{epoch+1}.pth"
#     torch.save({'state_dict': model.state_dict()}, checkpoint_path)
#     print(f"Saved checkpoint: {checkpoint_path}")


In [ ]:
# val_image_dir = "ProcessedDataV3/Validation_Set/Image"
# val_label_dir = "ProcessedDataV3/Validation_Set/Label"

# val_dataset = BrainSegmentationDataset(val_image_dir, val_label_dir)
# val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=2)

# model.eval()
# val_loss = 0
# with torch.no_grad():
#     for images, labels in val_loader:
#         images, labels = images.to(device), labels.to(device)
#         with autocast():
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#         val_loss += loss.item()

# avg_val_loss = val_loss / len(val_loader)
# print(f"Validation Loss: {avg_val_loss:.4f}")
